# **LDA Topic Modeling using Yelp Data**

In [31]:
import pandas as pd

data = pd.read_csv('yelp_all_header.csv');
data_text = data[['text']]
data_text['index'] = data_text.index
documents = data_text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [32]:
len(documents)

10000

# **Data Preprocessing**

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

np.random.seed(2019)

In [34]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Lemmatize example**

In [35]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


**Stemmer example**

In [36]:
stemmer = SnowballStemmer('english')

original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']

singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


**LDA Functions**

In [0]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [38]:
doc_sample = documents[documents['index'] == 1000].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Great', 'atmosphere', 'with', 'interesting', 'lights', '(look', 'like', 'white', 'jellyfish).', 'Got', 'a', 'Chicken', 'piccata', 'which', 'was', 'delicious.', 'TVs', 'were', 'conveniently', 'placed', 'so', 'we', 'were', 'able', 'to', 'watch', 'the', 'World', 'Series', 'Games.', 'Overall,', 'a', 'fun', 'place', 'to', 'catch', 'up', 'with', 'friends.']


 tokenized and lemmatized document: 
['great', 'atmospher', 'interest', 'light', 'look', 'like', 'white', 'jellyfish', 'chicken', 'piccata', 'delici', 'conveni', 'place', 'abl', 'watch', 'world', 'seri', 'game', 'overal', 'place', 'catch', 'friend']


In [0]:
processed_docs = documents['text'].map(preprocess)

In [40]:
processed_docs[:10]

0    [wife, take, birthday, breakfast, excel, weath...
1    [idea, peopl, review, place, go, probabl, grip...
2       [love, gyro, plate, rice, good, candi, select]
3    [rosi, dakota, love, chaparr, park, conveni, s...
4    [general, manag, scott, petello, good, assur, ...
5    [quiessenc, simpli, beauti, window, earthi, wo...
6    [drop, drive, food, good, cute, littl, green, ...
7    [luckili, travel, connect, flight, thank, phoe...
8    [definit, come, happi, hour, price, amaz, sake...
9    [nobuo, show, uniqu, talent, menu, care, craft...
Name: text, dtype: object

# **Bag of Words on the Dataset**

In [0]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [42]:
count = 0

for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 absolut
1 amaz
2 arriv
3 best
4 better
5 birthday
6 blend
7 bloodi
8 bread
9 breakfast
10 busi


In [0]:
dictionary.filter_extremes(no_below=1000, no_above=0.5, keep_n=100000)

In [52]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[1000]

[(3, 1), (6, 1), (7, 1), (10, 2), (16, 1), (19, 1), (43, 1)]

In [53]:
bow_doc_1000 = bow_corpus[1000]

for i in range(len(bow_doc_1000)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1000[i][0], 
                                                     dictionary[bow_doc_1000[i][0]], 
                                                     bow_doc_1000[i][1]))

Word 3 ("delici") appears 1 time.
Word 6 ("like") appears 1 time.
Word 7 ("look") appears 1 time.
Word 10 ("place") appears 2 time.
Word 16 ("friend") appears 1 time.
Word 19 ("great") appears 1 time.
Word 43 ("chicken") appears 1 time.


# **TF-IDF**

In [0]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [0]:
corpus_tfidf = tfidf[bow_corpus]

In [56]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.40443267992112153),
 (1, 0.2286434786803883),
 (2, 0.1593395590868908),
 (3, 0.23962071100536633),
 (4, 0.10262241316663322),
 (5, 0.25168787840204454),
 (6, 0.11557899822528042),
 (7, 0.3970515166132741),
 (8, 0.23012218249113142),
 (9, 0.16773682469910764),
 (10, 0.08687578313066219),
 (11, 0.4349383194577555),
 (12, 0.2535077807555187),
 (13, 0.24968831924142562),
 (14, 0.2281262172802218)]


# **Running LDA using Bag of Words**

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=2, id2word=dictionary, passes=10, workers=2)

In [60]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.064*"food" + 0.046*"like" + 0.042*"order" + 0.041*"time" + 0.041*"come" + 0.038*"servic" + 0.035*"go" + 0.033*"friend" + 0.028*"know" + 0.028*"best"
Topic: 1 
Words: 0.127*"place" + 0.108*"good" + 0.061*"great" + 0.051*"like" + 0.048*"love" + 0.044*"food" + 0.037*"time" + 0.034*"nice" + 0.030*"chicken" + 0.030*"pretti"


# **Running LDA using TF-IDF**

In [0]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=2, id2word=dictionary, passes=2, workers=4)

In [23]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.006*"good" + 0.006*"love" + 0.006*"place" + 0.006*"great" + 0.006*"food" + 0.005*"like" + 0.005*"time" + 0.005*"servic" + 0.005*"restaur" + 0.004*"think"
Topic: 1 Word: 0.007*"food" + 0.006*"great" + 0.006*"good" + 0.005*"pizza" + 0.005*"place" + 0.005*"friend" + 0.005*"time" + 0.004*"order" + 0.004*"like" + 0.004*"come"


# **Classification of the Topics**

**Performance evaluation by classifying sample document using LDA Bag of Words model**

In [24]:
processed_docs[1000]

['want',
 'california',
 'fanci',
 'cuisin',
 'place',
 'seek',
 'home',
 'countri',
 'cookin',
 'spot',
 'cracker',
 'barrel',
 'california',
 'lunch',
 'great',
 'good',
 'servic',
 'busi',
 'meal',
 'exact',
 'order',
 'choic',
 'cracker',
 'barrel',
 'food',
 'price',
 'unbeliev',
 'reason',
 'inexpens',
 'fast',
 'food',
 'restaur',
 'server',
 'super',
 'extrem',
 'courteous',
 'train']

In [26]:
for index, score in sorted(lda_model[bow_corpus[1000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9528924226760864	 
Topic: 0.021*"food" + 0.017*"place" + 0.015*"good" + 0.015*"like" + 0.013*"great" + 0.011*"order" + 0.009*"love" + 0.009*"come" + 0.008*"think" + 0.007*"restaur"

Score: 0.047107577323913574	 
Topic: 0.019*"good" + 0.018*"place" + 0.017*"time" + 0.015*"like" + 0.012*"great" + 0.012*"food" + 0.010*"go" + 0.009*"friend" + 0.009*"come" + 0.009*"servic"


**Performance evaluation by classifying sample document using LDA TF-IDF model**

In [27]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.9288648366928101	 
Topic: 0.007*"food" + 0.006*"great" + 0.006*"good" + 0.005*"pizza" + 0.005*"place" + 0.005*"friend" + 0.005*"time" + 0.004*"order" + 0.004*"like" + 0.004*"come"

Score: 0.07113512605428696	 
Topic: 0.006*"good" + 0.006*"love" + 0.006*"place" + 0.006*"great" + 0.006*"food" + 0.005*"like" + 0.005*"time" + 0.005*"servic" + 0.005*"restaur" + 0.004*"think"


**Testing model on unseen document**

In [29]:
unseen_document = 'That movie was terrible! I would hate to see it again.' #Negative sentiment
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8187199234962463	 Topic: 0.019*"good" + 0.018*"place" + 0.017*"time" + 0.015*"like" + 0.012*"great"
Score: 0.18128010630607605	 Topic: 0.021*"food" + 0.017*"place" + 0.015*"good" + 0.015*"like" + 0.013*"great"


In [30]:
unseen_document = 'That movie was amazing! I would love to see it again.' #Positive sentiment
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5849071741104126	 Topic: 0.021*"food" + 0.017*"place" + 0.015*"good" + 0.015*"like" + 0.013*"great"
Score: 0.4150928556919098	 Topic: 0.019*"good" + 0.018*"place" + 0.017*"time" + 0.015*"like" + 0.012*"great"
